<a href="https://colab.research.google.com/github/federicasibilla/ScenicOrNot/blob/main/Preparazione_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Scenic Or Not

## Dataset di immagini dal gioco https://scenicornot.datasciencelab.co.uk/

Il gioco propone immagini di scene outdoor scattate in Inghilterra, Scozia e Galles. Gli utenti possono assegnare ad ogni scena un punteggio intero da 1 a 10. Il dataset generato è composto da 7 colonne:


*   ID della fotografia
*   Latitudine
*   Longitudine
*   Punteggio medio
*   Varianza 
*   Elenco dei voti
*   URL per la pagina di http://www.geograph.org.uk/ contenente l'immagine

In questo notebook si implementa la creazione degli array utili allo studio del problema di riconoscere la scenicità di un luogo outdoor:



*   Array contenente gli array RGB delle immagini che sono state votate più di 10 volte
*   Array delle corrispondenti distribuzioni dei voti ricevuti per ciascuna immagine

Riportiamo un esempio di riga del dataset:

---

*39	 53.9849  	-1.41281  	3.6000  	1.8400	  5,4,4,3,1,6,3,2,4,4	 http://www.geograph.org.uk/photo/324*

---
negli array saranno contenuti, rispettivamente, l'immagine (riscalata a 224x224 pixel) contenuta nella pagina del link e l'array [0.1, 0.1, 0.2, 0.4, 0.1, 0.1, 0.0, 0.0, 0.0, 0.0].




